# Proyecto ML. Estimación de fallos de componentes en los bogies de un tren

## Objetivo General

El propósito de este proyecto es desarrollar un modelo de Machine Learning capaz de predecir fallos en los bogies de un tren con antelación. Anticipar estas averías permite mejorar la seguridad operativa, optimizar los planes de mantenimiento y reducir paradas imprevistas.

## ¿Por qué los bogies?

El bogie es uno de los componentes más críticos del tren. Es el conjunto que incluye las ruedas, la suspensión, los rodamientos y parte del sistema de frenado. Un fallo en el bogie puede provocar desde una reducción del confort hasta un accidente grave. Por ello, disponer de un modelo predictivo es un caso realista y de alto impacto en el sector ferroviario.

![Imagen_bogie](../data/img/Railroad_truck,_FM55-20.Fig8-8_es.png)

## Descripción del Dataset

Para abordar el problema, tratamos un dataset que contiene medidas de sensores
Este dataset contiene:

-  Medidas físicas

Variables como temperatura, vibraciones, presión o carga en rueda, que se relacionan directamente con el desgaste mecánico.

-  Indicadores operativos

Velocidad, kilometraje o condiciones ambientales que ayudan a contextualizar el estado del bogie.

 -  Metadatos

Información de mantenimiento y estado de los sensores, fundamentales para explicar anomalías o ruido en los datos.

-  Etiqueta objetivo

target_fault: indica si se produjo un fallo .

## Datasets de origen


In [1]:
import pandas as pd

In [7]:
df_1 = pd.read_csv("../data/bogies_dataset.csv")
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200187 entries, 0 to 200186
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   temperature         200187 non-null  object 
 1   pressure            200187 non-null  object 
 2   vibration           200187 non-null  float64
 3   wheel_load          200187 non-null  float64
 4   speed               200187 non-null  float64
 5   ambient_temp        200187 non-null  float64
 6   humidity            200187 non-null  int64  
 7   mileage             200187 non-null  float64
 8   bogie_id            195577 non-null  object 
 9   sensor_status       200187 non-null  object 
 10  maintenance_status  200187 non-null  object 
 11  failure_next_48h    200187 non-null  int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 18.3+ MB


In [8]:
df_1.head(10)

,temperature,pressure,vibration,wheel_load,speed,ambient_temp,humidity,mileage,bogie_id,sensor_status,maintenance_status,failure_next_48h
0,58.66,5.82,0.410116,9427.582278,106.917698,25.644634,48,264402.153977,B17,warn,overdue,0
1,42.52,5.21,0.443238,8980.860565,110.804648,18.213796,67,224296.350412,B20,ok,recent,0
2,64.01,4.02,0.179579,11500.221948,135.979812,21.773962,87,268242.546956,B7,ok,due,0
3,66.29 ºC,6.98 bar,0.345991,8041.624728,129.296741,26.117750,38,234359.259162,B28,ok,recent,0
4,31.59,6.53,0.429938,9727.319299,119.431576,26.468030,38,286410.414369,B19,ok,overdue,0
5,39.37,5.06,0.318579,10741.556265,120.789694,15.438115,41,274017.467672,B40,warn,RECENT,0
6,56.53,6.18,0.195330,9304.041587,128.861963,29.905412,48,259242.915054,B18,ok,due,0
7,51.21,4.77,0.132360,9715.049812,132.055451,18.296788,26,231372.418656,B22,ok,due,0
8,54.80,6.75,0.278186,9030.725752,105.446712,25.699204,66,148503.219863,B28,ok,due,0
9,44.76,7.16 bar,0.455728,10972.775728,125.904425,11.356556,77,282820.356649,B37,ok,recent,0


| **Columna**          | **Descripción**                                                                                                                                                        |
| -------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `temperature`        | Temperatura del rodamiento principal del bogie. Valores altos pueden indicar fricción excesiva, falta de lubricación o desgaste interno.                               |
| `pressure`           | Presión del sistema neumático del bogie (frenos y circuitos auxiliares). Presiones anómalas anticipan fallos de frenado o fugas.                                       |
| `vibration`          | Nivel de vibración estructural medido por acelerómetros. Es uno de los indicadores más predictivos de fallos: deformaciones, rodamientos dañados o ruedas defectuosas. |
| `wheel_load`         | Carga soportada por la rueda/eje. Una distribución irregular puede revelar problemas en suspensión o deformaciones.                                                    |
| `speed`              | Velocidad instantánea del tren, necesaria para contextualizar otras señales (vibración, temperatura, carga).                                                           |
| `ambient_temp`       | Temperatura exterior. Permite diferenciar una anomalía real del bogie de un simple día extremadamente caluroso.                                                        |
| `humidity`           | Humedad ambiental. Puede afectar a frenos, a sensores y al desgaste por corrosión.                                                                                     |
| `mileage`            | Kilómetros recorridos por el bogie desde la última revisión profunda. Variable clave para modelar degradación.                                                         |
| `bogie_id`           | Identificador único del bogie. Permite análisis por unidad y detección de bogies sistemáticamente problemáticos.                                                       |
| `sensor_status`      | Estado del sensor (OK / degradado / error). Afecta a la fiabilidad de las señales registradas.                                                                         |
| `maintenance_status` | Estado de mantenimiento reciente (none / minor / major). Una intervención reciente puede resetear patrones o introducir variaciones.                                   |
| `failure_next_48h`   | **Target** — Indica si el bogie falló en las siguientes 48 horas (0: no falla, 1: sí falla).                                                                           |


In [9]:
df_2 = pd.read_csv("../data/train_bogie_dataset.csv")

In [10]:
df_2.head()

,timestamp,train_id,bogie_id,operation_mode,track_condition,curve_class,track_gradient,speed_kmh,load_tons,external_temp_c,...,vibration_y_rms,vibration_z_rms,bogie_temp_c,wheel_temp_left_c,wheel_temp_right_c,bogie_health_score,fault_type,target_fault,alarm_level,days_since_last_maint
0,2024-01-01 00:00:00,7,21,service,good,straight,5.566133,111.181583,11.643473,11.140841,...,1.012804,1.444480,26.526055,26.083420,28.386208,0.060155,0.0,0.0,0,0
1,2024-01-01 00:01:00,20,22,service,fair,straight,2.954796,72.605862,18.075376,8.399445,...,2.087751,2.830601,44.982085,46.529224,47.451861,0.550271,0.0,0.0,0,0
2,2024-01-01 00:02:00,15,18,service,good,straight,-10.060349,103.341719,19.016757,11.734574,...,1.750199,2.300564,40.154026,40.966406,39.786704,0.332305,0.0,0.0,0,0
3,2024-01-01 00:03:00,11,9,service,good,sharp_curve,-3.587018,67.341700,15.225973,7.953995,...,0.923177,1.423216,23.853490,26.413339,21.196550,0.183175,0.0,0.0,0,0
4,2024-01-01 00:04:00,8,32,service,good,straight,1.878557,105.348183,11.883568,7.050180,...,1.661964,2.383944,28.120544,25.173249,29.493783,0.226758,0.0,0.0,0,0


In [11]:
df_2["target_fault"].value_counts()

target_fault
0.0    194049
1.0      2951
Name: count, dtype: int64

| Columna                  | Descripción                                   |
|--------------------------|-----------------------------------------------|
| timestamp                | Fecha/hora de medición                        |
| train_id                 | Identificador del tren                        |
| bogie_id                 | Identificador del bogie (1-40)                |
| operation_mode           | Modo: service/shunting/maintenance_move       |
| track_condition          | Estado vía: good/fair/poor                    |
| curve_class              | Tipo curva: straight/gentle/sharp             |
| track_gradient           | Pendiente de la vía                           |
| speed_kmh                | Velocidad del tren                            |
| load_tons                | Carga transportada por bogie                  |
| external_temp_c          | Temperatura ambiente                          |
| humidity_pct             | Humedad relativa exterior                     |
| days_since_inspection    | Días desde última inspección/                 |
| vibration_x_rms          | Vibración eje X (RMS)                         |
| vibration_y_rms          | Vibración eje Y (RMS)                         |
| vibration_z_rms          | Vibración eje Z (RMS)                         |
| bogie_temp_c             | Temperatura general bogie                     |
| wheel_temp_left_c        | Temp. rueda izquierda                         |
| wheel_temp_right_c       | Temp. rueda derecha                           |
| bogie_health_score       | Salud interna bogie (0=bueno, 1=malo)         |
| fault_type               | 0=normal, 1=warning, 2=fault, 3=critical      |
| target_fault             | Binario: 0=normal, 1=fallo                    |
| alarm_level              | 0=normal, 1=warning, 2=critical               |
| days_since_last_maint    | Días desde último mantenimiento real          |


## Variable Objetivo (Target)

- **`target_fault`** (binario 0/1): indica si hay algún fallo en el bogie  
- Aproximadamente 1.5% de los registros indican fallo  


In [13]:
df_3 = pd.read_csv("../data/bogie_maintenance_events.csv")
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   bogie_id          146 non-null    int64 
 1   maintenance_date  146 non-null    object
 2   maintenance_type  146 non-null    object
 3   downtime_hours    146 non-null    int64 
 4   cost_euros        146 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 5.8+ KB
